<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/langgraph_nodes_edges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install -q -U langchain langchain_openai langgraph google-search-results

In [82]:
import os
from google.colab import userdata

In [83]:
os.environ['SERPAPI_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

In [84]:
from langchain_community.utilities import SerpAPIWrapper

In [85]:
search = SerpAPIWrapper()

search.run("Obama's first name?")

'Barack Hussein Obama II'

In [86]:
import functools, operator, requests, os, json
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict

In [87]:
llm = ChatOpenAI(model="gpt-4-turbo-preview")

In [88]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage
)

@tool("web_search")
def web_search(query: str) -> str:
    """Search with Google SERP API by a query"""
    search = SerpAPIWrapper()
    return search.run(query)

@tool("twitter_writer")
def write_tweet(content: str) -> str:
    """Based a piece of content, write a tweet."""
    chat = ChatOpenAI()
    messages = [
      SystemMessage(
          content="You are a Twitter account operator."
                  " You are responsible for writing a tweet based on the content given."
                  " You should follow the Twitter policy and make sure each tweet has no more than 140 characters."
      ),
      HumanMessage(
          content=content
      ),
    ]
    response = chat(messages)
    return response.content

In [90]:
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

In [91]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [92]:
members = ["Search_Engine", "Twitter_Writer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [93]:
search_engine_agent = create_agent(llm, [web_search], "You are a web search engine.")
search_engine_node = functools.partial(agent_node, agent=search_engine_agent, name="Search_Engine")

twitter_operator_agent = create_agent(llm, [write_tweet], "You are responsible for writing a tweet based on the content given.")
twitter_operator_node = functools.partial(agent_node, agent=twitter_operator_agent, name="Twitter_Writer")

workflow = StateGraph(AgentState)
workflow.add_node("Search_Engine", search_engine_node)
workflow.add_node("Twitter_Writer", twitter_operator_node)
workflow.add_node("supervisor", supervisor_chain)

In [94]:
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [95]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Write a tweet about LangChain news")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Search_Engine'}}
----
{'Search_Engine': {'messages': [HumanMessage(content="🚀 Exciting news from LangChain! 🌟\n\nWe just launched LangGraph, a revolutionary tool to customize your Agent Runtime, marking a significant milestone in our journey. Also, we're thrilled to announce the release of langchain 0.1.0, our first stable version that's fully backward compatible. 🎉\n\nStay tuned for more updates on how we're transforming the AI ecosystem. #LangChain #Innovation #AI\n\n[Week of 1/22/24]", name='Search_Engine')]}}
----
{'supervisor': {'next': 'Twitter_Writer'}}
----
{'Twitter_Writer': {'messages': [HumanMessage(content='🚀 Exciting news from LangChain! 🌟 Introducing LangGraph, a customizable Agent Runtime tool, and langchain 0.1.0, our first stable release. #LangChain #Innovation #AI', name='Twitter_Writer')]}}
----
{'supervisor': {'next': 'FINISH'}}
----
